In [1]:
from pyairtable import Api
from pyairtable.formulas import match
import pandas as pd
import requests
import json
from collections import defaultdict
from turfpy.transformation import union
from geojson import FeatureCollection, Feature

api_airtable_key = "patfZxFRQiBjvIvvo.677f2eac18aacda776f61eace4bd46c28c6c393db89afd086a41933bd2f39ad2"
MAPBOX_TOKEN = "pk.eyJ1IjoiYXJ0aHVyY2hpcXVldCIsImEiOiJjbHBoaHB6MncwMXNlMmxucDBlYndpaW93In0.Muo3cgXaUJjMC14H9bXfUw"

In [2]:
colonnes_fiches = [
    "codecommune",
    "nomcommune",
    "sectioncadastrale",
    "numeroplan",
    "lieuxdit",
    "nomproprietaire",
    "modedefairevaloir",
    "nomcepage",
    "nomportegreffe",
    "anneeplantation",
    "surfaceplantee"
]

In [3]:
def match_surface_p(x, y, dict_surface_p):
    try:
        return dict_surface_p[x]
    except KeyError:
        return y / 10000

In [4]:
def formater_fiche(df):
    df = (
        df.groupby(["codecommune", "sectioncadastrale", "numeroplan"])
        .agg(
            {
                "nomproprietaire": "first",
                "nomcommune": "first",
                "lieuxdit": "first",
                "nomcepage": list,
                "anneeplantation": list,
                "surfaceplantee": "sum",
            }
        )
        .reset_index()
    )

    df["codecommune"]=df["codecommune"].astype(str).map(lambda x: x[:2]+x[3:])

    df["sectioncadastrale"] = (
        df["sectioncadastrale"]
        .map(lambda x: x.replace(" ", ""))
        .map(lambda x: (2 - len(x)) * "0" + x)
    )

    df["numeroplan"] = (
        df["numeroplan"].astype(str).map(lambda x: (4 - len(x)) * "0" + x)
    )

    df["nomcepage"] = df["nomcepage"].map(lambda x: str(list(set(x))))
    df["anneeplantation"] = df["anneeplantation"].map(lambda x: str(list(set(x))))
    df["id"] = df["codecommune"] + df["sectioncadastrale"] + df["numeroplan"]
    return df

In [5]:
def get_features(df, nom):
  features = []
  for i in range(df.shape[0]):

      try:
          res = requests.get(
              f"https://apicarto.ign.fr/api/cadastre/parcelle?code_insee={df['codecommune'][i]}&section={df['sectioncadastrale'][i]}&numero={df['numeroplan'][i]}&source_ign=PCI"
          )
          res.raise_for_status()
          coordinates = res.json()["features"][0]["geometry"]["coordinates"][0]
      except requests.ConnectionError:
          break
      except requests.RequestException:
          break
      except IndexError:
          coordinates = [[]]
      except KeyError:
          coordinates = [[]]
      features.append({
              "id" : abs(hash(df["id"][i])),
              "type": "Feature",
              "geometry": {
                  "type": "Polygon",
                  "coordinates": coordinates,
              },
              "properties": {
                  "nomproprietaire": nom,
                  "codecommune": df["codecommune"][i],
                  "nom": df["lieuxdit"][i],
                  "cepage": df["nomcepage"][i],
                  "annee": df["anneeplantation"][i],
                  "surface_p": int(df["surfaceplantee"][i])
              }
          })
      print(f"{i+1}/{df.shape[0]} parcelle(s) téléchargée(s)")
  return features

In [6]:
def geojson_to_dict_by_commune(features):
    commune_dict = defaultdict(list)
    for feature in features:
        commune = feature.get("properties", {}).get("codecommune")
        if commune:
            commune_dict[commune].append(Feature(geometry=feature['geometry']))
    return dict(commune_dict)

In [7]:
api = Api(api_airtable_key)
table = api.table('app08SGfxSPaGDDEH', 'tbl0Iz3svkG3iY4Xo')
records = table.all(formula=match({"Statut": "Pending"}))

In [8]:
i = 1
# Traitement des enregistrements
for record in records:
    print(f"record {i}/{len(records)}")

    # Récupération des données
    csv_url = record['fields']['Parcellaire'][0]['url']
    nom = record['fields']['Nom']
    adresse = record['fields']['Adresse']

    # Lecture et nettoyage du fichier CSV
    df = pd.read_csv(csv_url, sep=";")
    df.columns = (
        df.columns.str.replace(" ", "")
        .str.replace("_", "")
        .str.replace("-", "")
    )

    # Validation du format des données
    if df.columns.tolist() != colonnes_fiches:
        print("Le format de données ne correspond pas. Veuillez vérifier l'intitulé des colonnes de votre fichier")
        break
    elif df.shape[0] == 0:
        print("Le fichier est vide")
        break

    # Traitement des données du fichier CSV
    clean_df = formater_fiche(df)

    # Téléchargement des parcelles
    features = get_features(clean_df, nom)
    geojson = {
      "type": "FeatureCollection",
      "features": features
    }

    with open(f'../parcelles/exploitation/{nom}.json', 'w') as f:
        json.dump(geojson, f)

    try:
        result = geojson_to_dict_by_commune(features)

        for commune, features in result.items():
            try:
                with open(f'../parcelles/commune/{commune}.json', 'r') as file:
                    data = json.load(file)
            except FileNotFoundError:
                data = {
                    "type": "FeatureCollection",
                    "features": []
                }

            # Union des features
            unified_features = union(FeatureCollection(features))
            unified_features['properties']['nom'] = nom
            unified_features['properties']['adresse'] = adresse
            unified_features['id'] = abs(hash(nom))


            # Ajouter à la liste des features existantes
            if isinstance(unified_features, list):
                data["features"] += unified_features
            else:
                data["features"].append(unified_features)

            print(f"Enregistrement du fichier ../parcelles/commune/{commune}.json")
            with open(f'../parcelles/commune/{commune}.json', 'w') as file:
                json.dump(data, file)

        table.update(record['id'], {"Statut": "Available"})
        print("Le fichier JSON des parcelles a été mis à jour avec succès.")
    except Exception as e:
        print(f"Erreur lors de la mise à jour des parcelles : {e}")

    i+=1

record 1/1
1/32 parcelle(s) téléchargée(s)
2/32 parcelle(s) téléchargée(s)
3/32 parcelle(s) téléchargée(s)
4/32 parcelle(s) téléchargée(s)
5/32 parcelle(s) téléchargée(s)
6/32 parcelle(s) téléchargée(s)
7/32 parcelle(s) téléchargée(s)
8/32 parcelle(s) téléchargée(s)
9/32 parcelle(s) téléchargée(s)
10/32 parcelle(s) téléchargée(s)
11/32 parcelle(s) téléchargée(s)
12/32 parcelle(s) téléchargée(s)
13/32 parcelle(s) téléchargée(s)
14/32 parcelle(s) téléchargée(s)
15/32 parcelle(s) téléchargée(s)
16/32 parcelle(s) téléchargée(s)
17/32 parcelle(s) téléchargée(s)
18/32 parcelle(s) téléchargée(s)
19/32 parcelle(s) téléchargée(s)
20/32 parcelle(s) téléchargée(s)
21/32 parcelle(s) téléchargée(s)
22/32 parcelle(s) téléchargée(s)
23/32 parcelle(s) téléchargée(s)
24/32 parcelle(s) téléchargée(s)
25/32 parcelle(s) téléchargée(s)
26/32 parcelle(s) téléchargée(s)
27/32 parcelle(s) téléchargée(s)
28/32 parcelle(s) téléchargée(s)
29/32 parcelle(s) téléchargée(s)
30/32 parcelle(s) téléchargée(s)
31/32 pa